In [0]:
import duckdb

import s3fs
from datetime import datetime

# 1. Conectar ao DuckDB
con = duckdb.connect()

# 2. Instalar e carregar extensão httpfs
con.execute("INSTALL httpfs;")
con.execute("LOAD httpfs;")

# 3. Configurar credenciais da AWS
con.execute("SET s3_region='us-east-1';")
con.execute("SET s3_access_key_id='AWS_ACCESS_KEY_ID';")
con.execute("SET s3_secret_access_key='AWS_SECRET_ACCESS_KEY';")


# 4. Ler Bronze do S3
con.execute("""
CREATE OR REPLACE TABLE bronze_data AS
SELECT * FROM read_parquet('s3://nasa-weather-bronze/bronze/nasa_weather/nasa_insight_2025-11-02.parquet');
""")

# 5. Criar Silver com limpeza
con.execute("""
CREATE OR REPLACE TABLE silver_data AS
SELECT
    sol,
    temp_avg AS temp_celsius,
    season
FROM bronze_data
WHERE sol IS NOT NULL AND temp_avg IS NOT NULL AND season IS NOT NULL;
""")

# 6. Exportar Silver para o S3
data_str = datetime.now().strftime("%Y-%m-%d")
silver_path = f"s3://nasa-weather-bronze/silver/nasa_weather/nasa_insight_silver_{data_str}.parquet"

fs = s3fs.S3FileSystem(
   aws_access_key_id = dbutils.secrets.get("aws", "access_key_id")
aws_secret_access_key = dbutils.secrets.get("aws", "secret_access_key")
    client_kwargs={'region_name': 'us-east-1'}
)

df_silver = con.execute("SELECT * FROM silver_data").fetchdf()
with fs.open(silver_path, 'wb') as f:
    df_silver.to_parquet(f, index=False)
